In [7]:
from submissions.ab_submission.object_detector import imageDataLoader, follicleClassifier, ObjectDetector

In [8]:
import glob
import pandas as pd
from matplotlib import pyplot as plt

train_files = glob.glob("./data/train/*.jpg")
test_files = glob.glob("./data/test/*.jpg")
train_label = pd.read_csv("./data/train/labels.csv")
test_label = pd.read_csv("./data/test/labels.csv")

In [12]:
model = ObjectDetector(ramp_mode=False)
model.load(boxPixelClassifier="./params/boxPixelClassifier_opencv.joblib", follicleClassifier="./params/follicleClassifier2.model")
model.fit(train_files, train_label)
#model.save(boxPixelClassifier="./params/boxPixelClassifier_opencv.joblib",follicleClassifier="./params/follicleClassifier2.model")

Model loaded from ./params/follicleClassifier2.model
Fitting boxPixelClassifier
Fitting follicleClassifier


C:\Users\4078182\Miniconda3\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
test_loader = imageDataLoader(test_files, test_label[["filename","xmin","xmax","ymin","ymax","label"]])
train_loader = imageDataLoader(train_files, train_label[["filename","xmin","xmax","ymin","ymax","label"]])

In [14]:
y_hat = model.predict(test_files)

TypeError: dict expected at most 1 argument, got 2

In [11]:
from PIL import Image, ImageDraw, ImageFont

font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", 60, encoding="unic")

def write_rectangle(image, bbox, labels, folder=None, filename=None):
    img = Image.fromarray(image)
    img_draw = ImageDraw.Draw(img)
    for box,label in zip(bbox, labels):
        x1, y1, x2, y2 = box
        img_draw.rounded_rectangle(((x1, y1), (x2,y2)), fill=None, outline="black", width=5)
        img_draw.text((x1, y1-70), label, font=font, fill="black")

    if folder is not None and filename is not None:
        img.save(f"./data/{folder}/{filename}")
    
    return np.array(img)

In [12]:
i = 0
folder = "test_predicted"
for x in test_loader.get_samples():
    write_rectangle(x[0], y_hat[i]["bbox"], y_hat[i]["class"], folder=folder, filename=test_loader.X_filenames[i])
    i += 1    